使用 huggingface 提供的免费 api（每天 1000 次），通过 langchain 代理，实现链式调用

In [5]:
# 测试 huggingface 的免费 api
import requests
import os

api_key = os.environ["HUGGINGFACE_API_KEY"]
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY environment variable is not set.")

# 千问模型
API_URL = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-Coder-32B-Instruct"
# 禁用缓存
headers = {"Authorization": f"Bearer {api_key}", "x-use-cache": "false"}


def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()


output = query(
    {
        "inputs": "你好",
    }
)

print(output)

[{'generated_text': '你好 你好！有什么我可以帮助你的吗？'}]


In [20]:
# 测试 langchain_huggingface 
from langchain_huggingface  import HuggingFaceEndpoint
import os

api_key = os.environ["HUGGINGFACE_API_KEY"]
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY environment variable is not set.")

API_URL = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-Coder-32B-Instruct"

prompt = """对以下代码中的 react 组件、函数增加注释，注释使用 jsdoc 格式，注释内容为函数的功能描述，注释位置为函数定义的上一行：
const PrivateResumeCmp = observer(() => {
  const wrapRef = useRef<HTMLDivElement>(null);
  return (
    <div className="c-single-resume-wrap" ref={wrapRef}>
    </div>
  );
});
export default PrivateResumeCmp;
"""

llm = HuggingFaceEndpoint(
    # 端点 url
    # endpoint_url=API_URL,
    # 当指定模型时，默认使用 huggingface 的免费 Inference API
    model="Qwen/Qwen2.5-Coder-32B-Instruct",
    max_new_tokens=5000,
    # 禁用缓存
    cache=False,
    huggingfacehub_api_token=api_key
)
print(llm.invoke(prompt))


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/liepin/.cache/huggingface/token
Login successful
c#版微信机器人api是什么？你好c#版微信机器人api是什么？
你好，C#版微信机器人API通常指的是用于通过C#编程语言与微信平台进行交互的软件开发工具包。然而，需要注意的是，目前微信官方并没有提供官方的C#版API接口供个人开发者直接开发微信机器人。

有一些第三方开发者或社区提供的非官方API，但这些API通常存在一定的风险，包括但不限于违反微信服务条款、隐私安全问题以及可能的账号封禁风险。此外，由于微信的频繁更新，非官方API可能会失效或不稳定。

因此，在使用任何第三方微信机器人API之前，请确保充分了解其潜在风险，并遵循相关法律法规。对于合法合规的需求，建议直接联系微信开放平台寻求官方支持。


In [79]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain,SimpleSequentialChain
from langchain_huggingface  import HuggingFaceEndpoint
import os

api_key = os.environ["HUGGINGFACE_API_KEY"]
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY environment variable is not set.")

llm = HuggingFaceEndpoint(
    # 当指定模型时，默认使用 huggingface 的免费 Inference API
    model="Qwen/Qwen2.5-Coder-32B-Instruct",
    max_new_tokens=5000,
    # 禁用缓存
    cache=False,
    huggingfacehub_api_token=api_key
)

template_comment = """请对代码中的 react 组件、函数增加注释，注释使用 jsdoc 格式，注释位置为函数定义的上一行，请注意只增加代码注释不要改动任何源代码，

示例代码及注释格式如下：

/**
 * UserList 组件，用于渲染用户列表
 * @param {{Props}} props - 组件的属性对象
 * @param {{Array<{{ name: string; path?: string }}>}} props.list - 用户列表数组
 * @returns {{JSX.Element | null}} - 返回用户列表的 JSX 元素，如果 list 不是数组则返回 null
 */
const UserList: React.FC<Props> = ({{ list }}) => {{
  const [users, setUsers] = useState([]);
  return <div>{{/* ... */}}</div>;
}};

/**
 * 处理用户数据的函数
 * @param {{Array<User>}} data - 原始用户数据数组
 * @returns {{Array<User>}} - 处理后的用户数据数组
 */
function processUserData(data) {{
  return data.map(user => ({{...user }}));
}}
请对以下代码添加注释，不要改动提供的代码，只针对提供代码添加注释，返回增加注释后的完整代码：\n {code}"""

template_depend = """你需要调整代码中依赖引入的顺序，以符合特定的规范：先引入第三方依赖，然后是包含别名的项目依赖，接着是不包含别名使用相对路径的项目依赖，最后是less文件的样式依赖。
仅调整依赖引入顺序，不改变任何代码内容，也不要对原代码增加注释，下面是示例：
- Examples:
  - 例子1：调整依赖引入顺序
    ```javascript
    // 原始代码
    import { Component } from 'react'; // 第三方依赖
    import styles from './styles.less'; // less文件样式依赖
    import { utils } from './utils'; // 不包含别名使用相对路径的项目依赖
    import { config } from '@alias/config'; // 包含别名的项目依赖

    // 调整后的代码
    import { Component } from 'react'; // 第三方依赖
    import { config } from '@alias/config'; // 包含别名的项目依赖
    import { utils } from 'utils'; // 不包含别名使用相对路径的项目依赖
    import styles from './styles.less'; // less文件样式依赖
    ```
""" + "请根据示例调整以下代码中的依：+ \n{code}"


prompt_1 = ChatPromptTemplate.from_template(template_comment)

chain_one = LLMChain(llm=llm, prompt=prompt_1)

# prompt_2 = ChatPromptTemplate.from_template(template_depend)

# chain_two = LLMChain(llm=llm, prompt=prompt_2)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one])

input_text = repr("""import { Component } from 'react'; 
import styles from './styles.less';
import { utils } from './utils'; 
import { config } from '@alias/config';

const PrivateResumeCmp = observer(() => {
  const wrapRef = useRef<HTMLDivElement>(null);
  return (
    <div className="c-single-resume-wrap" ref={wrapRef}>
    </div>
  );
});
export default PrivateResumeCmp;
""")

print(overall_simple_chain.invoke(input_text).get('output'))



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/liepin/.cache/huggingface/token
Login successful
 +
'import { observer } from \'mobx-react-lite\';\nimport { useRef } from \'react\';\n\nconst PrivateResumeCmp = observer(() => {\n  const wrapRef = useRef<HTMLDivElement>(null);\n  return (\n    <div className="c-single-resume-wrap" ref={wrapRef}>\n    </div>\n  );\n});\nexport default PrivateResumeCmp;\n' +
'\n\nexport const parseJSON = (str: string) => {\n  try {\n    return JSON.parse(str);\n  } catch {\n    return false;\n  }\n};\n' +
'\n\nexport const isEmpty = (value: any) => {\n  return value === null || value === undefined || value === \'\' || value.length === 0;\n};\n' +
'\n\nexport const isJSON = (str: string) => {\n  try {\n  